In [194]:
import requests
import pandas as pd
import time

In [195]:
headers = {'user_agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'}
url = 'https://huiyan.baidu.com/migration/cityrank.json?dt=city&id=420100&type=move_out&date=20200314'#这里使用json而不是jsonp，否则存在非json格式的数据，会报错
r = requests.get(url,headers=headers)

In [196]:
print(type(r.text))

<class 'str'>


In [197]:
import json
data_json = json.loads(r.text)
data_json.keys()

dict_keys(['errno', 'errmsg', 'data'])

In [198]:
data = data_json['data']['list']
data[:5]

[{'city_name': '黄冈市', 'province_name': '湖北省', 'value': 21.15},
 {'city_name': '孝感市', 'province_name': '湖北省', 'value': 14.94},
 {'city_name': '咸宁市', 'province_name': '湖北省', 'value': 7.35},
 {'city_name': '荆州市', 'province_name': '湖北省', 'value': 7.14},
 {'city_name': '恩施土家族苗族自治州', 'province_name': '湖北省', 'value': 6.55}]

In [199]:
data[0].keys()

dict_keys(['city_name', 'province_name', 'value'])

In [200]:
pd.DataFrame(data).head()

,city_name,province_name,value
0,黄冈市,湖北省,21.15
1,孝感市,湖北省,14.94
2,咸宁市,湖北省,7.35
3,荆州市,湖北省,7.14
4,恩施土家族苗族自治州,湖北省,6.55


In [201]:
lastData = pd.DataFrame(data)

In [202]:
def save_data(data,name):
    file_name = name+'_'+time.strftime('%Y_%m_%d',time.localtime(time.time()))+'迁出数据'+'.csv'
    data.to_csv(file_name,index=None,encoding='utf_8_sig')
    print(file_name+'保存成功！')

In [203]:
save_data(lastData,'武汉')

武汉_2021_05_09迁出数据.csv保存成功！


In [204]:

import requests
import json
import time
from pyecharts import options as opts
from pyecharts.charts import Geo,Timeline,Bar,Grid,Page
from pyecharts.globals import ChartType, SymbolType,ThemeType
from pyecharts.components import Table

timeline = Timeline(
        init_opts=opts.InitOpts(width="1300px", height="650px",theme=ThemeType.DARK)
    )
total_move_out = {}


headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.204 Safari/537.36',
}

#省级数据
base_url = 'https://huiyan.baidu.com/migration/provincerank.json?'

for i in range(20200110,20200124,1):
    if (i>20200131 and i<20200201) or (i>20200229 and i<20200301):
        pass
    else:
        params = {
            'dt': 'city',
            'id': '420100',
            'type': 'move_out',
            'date': i,
        }
        response = requests.get(base_url,params=params,headers=headers)
        # print(response)
        datas = json.loads(response.text)#json字符串转换为python类型
        # print(datas)
        move_out = {}
        for data in datas['data']['list']:
            province = data['province_name'] # 所属省份
            if '黑龙江' in province:
                province = province[0:3]
            elif '内蒙古' in province:
                province = province[0:3]
            else:
                province = province[0:2]
            ratio = data['value']#比例
            move_out[province] = ratio
            if i != 20200110:
                total_move_out[province]+=ratio
                
        if i == 20200110:
            total_move_out = move_out
           
            
        data = [(i,j) for i,j in move_out.items()]
        data_pair = [('武汉',j) for j in move_out.keys()]


        c = (
            Geo()
            .add_schema(
                maptype="china",
                itemstyle_opts=opts.ItemStyleOpts(color="#323c48", border_color="#111"),
            )
            .add(
                "",
                data,
                type_=ChartType.EFFECT_SCATTER,
                color = 'white',
            )
            .add(
                "geo",
                data_pair,
                type_=ChartType.LINES,
                effect_opts=opts.EffectOpts(
                    symbol=SymbolType.ARROW, symbol_size=6
                ),
                linestyle_opts=opts.LineStyleOpts(curve=0.2),
            )
            .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
            .set_global_opts(title_opts=opts.TitleOpts(title=str(i)+"武汉迁徙地图",pos_left="200",
                pos_top="top",))
        )
        
        bar_x_data = [x for x in move_out.keys()]
        bar_y_data = [{"name": i, "value": j} for i,j in move_out.items()]
        bar = (
        Bar()
        .add_xaxis(xaxis_data=bar_x_data)
        .add_yaxis(
            series_name="",
            y_axis=bar_y_data,
            label_opts=opts.LabelOpts(
                is_show=True, position="right", formatter="{b} : {c}"
            ),
        )
        .reversal_axis()
        .set_global_opts(
            xaxis_opts=opts.AxisOpts(
                max_=100, axislabel_opts=opts.LabelOpts(is_show=False)
            ),
            yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(is_show=False)),
            tooltip_opts=opts.TooltipOpts(is_show=False),
            visualmap_opts=opts.VisualMapOpts(
                is_calculable=True,
                dimension=0,
                pos_left="10",
                pos_top="top",
                range_text=["High", "Low"],
                range_color=["lightskyblue", "yellow", "orangered"],
                textstyle_opts=opts.TextStyleOpts(color="#ddd"),
                min_=0,
                max_=100,
            ),
        )
        )
        grid_chart = (
            Grid()
            .add(
                bar,
                grid_opts=opts.GridOpts(
                    pos_left="10", pos_right="45%", pos_top="40%", pos_bottom="5"
                ),
            )
            .add(c,grid_opts=opts.GridOpts())#grid_opts必须要明确写出来
        )
        timeline.add(grid_chart,time_point=i)


for i,j in total_move_out.items():
    total_move_out[i]/=14
    
print(total_move_out)

timeline.add_schema(
    orient="vertical",
    is_auto_play=True,
    is_inverse=True,
    play_interval=1000,
    pos_left="null",
    pos_right="5",
    pos_top="20",
    pos_bottom="20",
    width="90",
    label_opts=opts.LabelOpts(is_show=True, color="#fff"),
    )
timeline.render('武汉迁徙地图.html')

{'湖北': 68.32428571428571, '河南': 5.707142857142857, '湖南': 3.540714285714286, '广东': 1.9664285714285714, '安徽': 2.310714285714286, '江西': 2.164285714285714, '江苏': 1.5400000000000003, '北京': 0.9785714285714286, '浙江': 1.1300000000000001, '山东': 1.1485714285714284, '重庆': 1.3007142857142855, '四川': 1.2942857142857138, '河北': 0.9835714285714287, '上海': 0.737857142857143, '广西': 0.8414285714285714, '贵州': 0.6042857142857142, '福建': 0.9321428571428573, '陕西': 0.7435714285714285, '山西': 0.6150000000000001, '云南': 0.5578571428571429, '甘肃': 0.3678571428571428, '海南': 0.3814285714285715, '辽宁': 0.34928571428571425, '新疆': 0.2271428571428571, '内蒙古': 0.19, '黑龙江': 0.2935714285714286, '天津': 0.16428571428571423, '吉林': 0.17785714285714285, '宁夏': 0.09071428571428573, '青海': 0.062142857142857166, '西藏': 0.019285714285714288}


'C:\\Users\\admin\\Desktop\\jupyter notebook\\武汉迁徙地图.html'

In [215]:
table1_data = []
for key,value in total_move_out.items():
    table1_data.append([key,value])#对于字典向list转化
    
table1_data.sort(key = lambda x:x[1],reverse = True)
table1_data = table1_data[:10]
table1 = Table()
headers = ["省份","比例"]
rows = []
for data in table1_data:
    rows.append(data)
table1.add(headers,rows)

In [216]:
headers = {'user_agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'}
url = 'http://ncov.myvis.cn:8002/api/getTimeData?level=1&name=china'
cov_r =requests.get(url)

cov_data_json = json.loads(r.text)
cov_data_json.keys()
cov_data = cov_data_json['data']['2020-03-12']#注意变量命名问题
print(cov_data)

[[500000, 576, '重庆市', 0, 6, 554, 0, '2020-03-12 07:09:57'], [230000, 482, '黑龙江省', 0, 13, 440, 0, '2020-03-12 08:12:24'], [450000, 252, '广西壮族自治区', 0, 2, 241, 0, '2020-03-12 08:17:48'], [370000, 760, '山东省', 0, 6, 733, 0, '2020-03-12 08:31:51'], [430000, 1018, '湖南省', 0, 4, 998, 0, '2020-03-12 08:42:37'], [520000, 146, '贵州省', 0, 2, 137, 0, '2020-03-12 09:03:10'], [110000, 435, '北京市', 0, 8, 334, 0, '2020-03-12 09:09:39'], [420000, 67781, '湖北省', 0, 3056, 50298, 8, '2020-03-12 09:21:37'], [330000, 1215, '浙江省', 0, 1, 1197, 0, '2020-03-12 09:21:37'], [610000, 245, '陕西省', 0, 2, 232, 0, '2020-03-12 09:29:10'], [120000, 136, '天津市', 0, 3, 132, 0, '2020-03-12 09:35:37'], [810000, 129, '香港特别行政区', 0, 3, 67, 3, '2020-03-12 09:43:09'], [360000, 935, '江西省', 0, 1, 934, 0, '2020-03-12 10:08:05'], [510000, 539, '四川省', 0, 3, 495, 0, '2020-03-12 11:50:05'], [440000, 1356, '广东省', 0, 8, 1289, 0, '2020-03-12 11:50:05'], [460000, 168, '海南省', 0, 6, 160, 0, '2020-03-12 11:50:05'], [320000, 631, '江苏省', 0, 0, 629, 0,

In [217]:
table2_data = []
for province in cov_data:
    name = province[2]
    if '黑龙江' in name:
        name = name[0:3]
    elif '内蒙古' in name:
        name = name[0:3]
    else:
        name = name[0:2]
    table2_data.append([name,province[1]])
    

table2_data.sort(key = lambda x:x[1],reverse = True)
table2_data = table2_data[:10]

table2 = Table()
headers = ["省份","确诊人数"]
rows = []
for data in table2_data:
    rows.append(data)
    
table2.add(headers,rows)

In [218]:
page = Page(layout=Page.SimplePageLayout)
page.add(
    timeline,
    table1,
    table2,
)
page.render("迁徙和疫情关系.html")

'C:\\Users\\admin\\Desktop\\jupyter notebook\\迁徙和疫情关系.html'